# Datenbereinigung: Bearbeitung von Einträgen ohne Verkäufe

In der Datenaufbereitung müssen fehlerhafte oder fehlende Werte in der Spalte `Sales` behandelt werden. Diese Werte können nicht gelöscht werden, da dies die Kontinuität der Zeitreihe unterbrechen würde. Die Integrität der Zeitreihen ist jedoch entscheidend für die Genauigkeit und Zuverlässigkeit des Modells. Um die fehlenden `Sales`-Werte zu füllen, verwenden wir den Mittelwert der `Sales`-Werte der vier vorhergehenden Samstage. Diese Methode berücksichtigt wöchentliche Muster und erzeugt realistische Schätzungen für die fehlenden Datenpunkte. Durch die Berechnung des Mittelwerts aus den vorhergehenden gleichen Wochetagen reflektieren die eingefügten Werte die typischen Betriebsbedingungen des jeweiligen Wochentages und verhindern Verzerrungen in den Daten. Der Prozess der Datenbereinigung im Falle der fehlenden Einträge für Verkäufe umfasst daher die folgenden drei Schritte: die Identifizierung der fehlerhaften oder fehlenden `Sales`-Werte, die Berechnung des Mittelwerts der `Sales`-Werte der vier vorhergehenden gleichen Wochentagen, das Einfügen dieser berechneten Mittelwerte in die entsprechenden fehlenden Datenfelder. Durch diese sorgfältige Methode zur Datenbereinigung wird gewährleistet, dass die Kontinuität der Zeitreihe erhalten bleibt und die Integrität der Daten gewahrt wird. Dies ist entscheidend, um ein genaues und verlässliches Vorhersagemodell zu entwickeln. Die realistischen Schätzungen der fehlenden Werte tragen dazu bei, dass das Modell auf konsistenten und repräsentativen Daten basiert, was letztlich zu präziseren und verlässlicheren Vorhersagen führt. Die Spalte `Customers` wird nicht mit neuen Daten befüllt, da diese Spalte nicht als Feature für das Modells verwendet wird.

## Notwendigkeit für die Datenbearbeitung

1. **Fehlende Begründung**

In den Fällen, in denen ein Geschäft geöffnet war, jedoch keine Kunden verzeichnet wurden und keine Verkäufe stattfanden, liegt kein erkennbarer Grund vor, der diese Situation plausibel erklärt. Solche Einträge können auf Fehler in der Datenerfassung oder untypische Betriebsbedingungen hinweisen, die nicht repräsentativ für den normalen Geschäftsbetrieb sind.

2. **Vermeidung von Modellverwirrung**

Ein Modell zur Vorhersage zukünftiger Verkäufe basiert auf der Annahme, dass die zugrunde liegenden Daten realistische und sinnvolle Geschäftsszenarien widerspiegeln. Einträge mit geöffneten Geschäften ohne Kunden und Verkäufe könnten das Modell verwirren und die Vorhersagegenauigkeit beeinträchtigen. Diese Einträge würden das Modell möglicherweise dazu bringen, unlogische Muster zu erlernen, was die Qualität der Vorhersagen mindern würde.

Aufgrund der oben genannten Punkte wurden diese Einträge als Datenrauschen(Noise)/Fehlerhaft klassifiziert. Datenrauschen sind irrelevante oder irreführende Informationen, die die Leistungsfähigkeit des Modells verringern können.


In [1]:
import pandas as pd

In [2]:
file_store = "../data/store.csv"
file_train = "../data/train.csv"
file_test = "../data/test.csv"

In [3]:
df_store = pd.read_csv(file_store, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)
df_train = pd.read_csv(file_train, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)
df_test = pd.read_csv(file_test, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

In [4]:
df_merged = pd.merge(df_store, df_train, on='Store')

In [5]:
# Standardize StateHoliday column
df_merged['StateHoliday'] = df_merged['StateHoliday'].astype(str)

# Find and replace zero sales with the average of the same weekday for the past 4 weeks
def replace_zero_sales(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Weekday'] = df['Date'].dt.day_name()
    
    for index, row in df[(df['Sales'] == 0) & (df['Open'] == 1)].iterrows():
        weekday = row['Weekday']
        date = row['Date']
        past_dates = df[(df['Weekday'] == weekday) & (df['Date'] < date)].sort_values(by='Date', ascending=False).head(4)
        if not past_dates.empty:
            avg_sales = past_dates['Sales'].mean()
            df.at[index, 'Sales'] = avg_sales
    
    return df

# Remove entries with zero sales and store open, and replace with average sales
df_merged = replace_zero_sales(df_merged)

# Verify the entries are replaced
print("\nTrain Data after replacing zero sales with average sales:")
print(df_merged[(df_merged['Sales'] == 0) & (df_merged['Open'] == 1)])


C:\Users\schmu\AppData\Local\Temp\ipykernel_17412\516383344.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5706.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Sales'] = avg_sales



Train Data after replacing zero sales with average sales:
        Store StoreType Assortment  CompetitionDistance  \
331720    364         a          c              13620.0   
927132   1017         c          a                110.0   

        CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
331720                        NaN                       NaN       1   
927132                       11.0                    2008.0       0   

        Promo2SinceWeek  Promo2SinceYear     PromoInterval  DayOfWeek  \
331720             10.0           2014.0  Mar,Jun,Sept,Dec          3   
927132              NaN              NaN               NaN          4   

             Date  Sales  Customers  Open  Promo StateHoliday  SchoolHoliday  \
331720 2013-05-08    0.0          0     1      0            0              0   
927132 2014-06-05    0.0          0     1      1            0              0   

          Weekday  
331720  Wednesday  
927132   Thursday  


Der gegebene Code verarbeitet Verkaufsdaten eines Geschäfts, um sicherzustellen, dass Einträge mit Null Verkaufszahlen (`Sales`) und geöffnetem Geschäft (`Open`) gefunden und durch den Durchschnitt der Verkaufszahlen der vorangegangenen vier gleichen Wochentage ersetzt werden. Zunächst wird die `StateHoliday`-Spalte standardisiert, indem alle Werte in dieser Spalte in Strings umgewandelt werden. Dies stellt sicher, dass alle Werte konsistent als Zeichenketten behandelt werden. Anschließend wird die Funktion `replace_zero_sales` definiert und aufgerufen. Diese Funktion durchläuft die Daten, um alle Einträge zu finden, bei denen die Verkaufszahlen Null sind und das Geschäft geöffnet war. Zuerst wird die `Date`-Spalte in ein Datetime-Format konvertiert und eine neue Spalte `Weekday` hinzugefügt, die den Namen des Wochentages enthält. Für jeden Eintrag mit Null Verkaufszahlen wird der Durchschnitt der Verkaufszahlen der letzten vier gleichen Wochentage berechnet. Diese Berechnung berücksichtigt nur die Einträge vor dem aktuellen Datum des Eintrags und sortiert sie absteigend nach Datum, um die letzten vier Einträge zu erhalten. Nach der Berechnung und dem Einsetzen der durchschnittlichen Verkaufszahlen wird überprüft, ob alle Einträge mit Null Verkaufszahlen ersetzt wurden.

# Fehlende Werte Open-Spalte Test-Datensatz

Der Testdatensatz beinhaltet fehlende Einträge in der Spalte `Open`. Diese Werte werden ersetzt, indem überprüft wird, ob der Wochentag Montag bis Samstag ist und kein staatlicher Feiertag vorliegt. Wenn diese Bedingungen erfüllt sind, wird der Wert in der Spalte `Open` auf 1 gesetzt, das heißt der Store ist an diesem Tag geöffnet. Andernfalls wird der Wert auf 0 gesetzt.

In [6]:
# Convert 'Date' column to datetime
df_test['Date'] = pd.to_datetime(df_test['Date'])

# Create new columns for the day of the week and StateHoliday
df_test['DayOfWeek'] = df_test['Date'].dt.dayofweek  # Montag=0, Sonntag=6
df_test['StateHoliday'] = df_test['StateHoliday'].astype(str)

# Replace missing entries in the 'Open' column
df_test['Open'] = df_test.apply(lambda row: 1 if row['DayOfWeek'] < 6 and row['StateHoliday'] == '0' else 0 if pd.isna(row['Open']) else row['Open'], axis=1)

# Review of the changes
print(df_test[df_test['Open'].isna()])  # Should be empty if all NaN values have been replaced


Empty DataFrame
Columns: [Id, Store, DayOfWeek, Date, Open, Promo, StateHoliday, SchoolHoliday]
Index: []
